# Problema 1

$$X_k=\sum_{n=0}^{N-1} x_n e^{\frac{-2\pi i}{N}kn}$$

#### a) Codificar una función en Python que calcule la dft. Considerar que debe dar como argumentos de entrada un arreglo de números reales y otro de números imaginarios.
> En primer lugar, necesitamos realizar un cambio, de tal forma que la solución sea mucho más amigable Sea que existe un x_n, lo definimos con una componente real y otra imaginaria.

$$ x_n = a_n + j\cdot b_n$$

>Además será necesario conocer la definición de exponencial, si: 
 
$$ e^{j\theta} = \cos(\theta) + j \sin(\theta)$$

>Por lo que:

$$ e^{\frac{-2\pi i}{N}kn} =  \cos(\frac{2\pi}{N}kn) - j \sin(\frac{2\pi}{N}kn) $$

>Entonces:

$$\sum_{n=0}^{N-1} x_n e^{\frac{-2\pi i}{N}kn} = \sum_{n=0}^{N-1} (a_n + j\cdot b_n)\cdot(\cos(\frac{2\pi}{N}kn) - j \sin(\frac{2\pi}{N}kn))$$

>Finalmente:

$$ X_{real_k} = \sum_{n=0}^{N-1} (a_n\cdot \cos(\frac{2\pi}{N}kn) + b_n \cdot \sin(\frac{2\pi}{N}kn) )$$

$$ X_{imag_k} = \sum_{n=0}^{N-1} (b_n\cdot \cos(\frac{2\pi}{N}kn) - a_n \cdot \sin(\frac{2\pi}{N}kn) )$$

>Donde:

$$\sum_{n=0}^{N-1} x_n e^{\frac{-2\pi i}{N}kn} = X_{real_k} + j\cdot X_{imag_k}$$


In [1]:
%%file Pyfft.py
import ctypes
import numpy as np
import random
import math

def fftpy(real,imag):
    N = len(real)
    salida_real = []
    salida_imag = []
    for k in range(N):
        real_acumulado = 0
        imag_acumulado = 0
        for n in range(N):
            angle = 2*math.pi*k*n/N
            real_acumulado += real[n]*math.cos(angle) + imag[n]*math.sin(angle)
            imag_acumulado += imag[n]*math.cos(angle) - real[n]*math.sin(angle)
        salida_real.append(real_acumulado)
        salida_imag.append(imag_acumulado)
    return salida_real, salida_imag

Overwriting Pyfft.py


#### b) Codificar una función en C que calcule la dft.
                                        

In [2]:
%%file Cfft.c
#include <stdio.h>
#include <math.h>
#define PI (3.141592653589793)
void fftC(int N,double *real, double *imag, double *salida_real, double *salida_imag);

void fftC(int N,double *real, double *imag, double *salida_real, double *salida_imag){
    for(int k = 0; k < N; k++){
        double k_term = 0;
        double k_term_imag = 0;
        for(int n = 0; n < N; n++){
            k_term += real[n]*cos(2*PI*k*n/N) + imag[n]*sin(2*PI*k*n/N);
            k_term_imag += imag[n]*cos(2*PI*k*n/N) - real[n]*sin(2*PI*k*n/N);
        }
        salida_real[k] = k_term;
        salida_imag[k] = k_term_imag;
    }
}

Overwriting Cfft.c


#### c) Codificar una función en ASM que calcule la dft.


In [507]:
%%file ASMfft.asm
global ASMfft

section .text
; xmm0
;r8 <-  salida_imag[0]  
;rcx <- salida_real[0]
;rdx <- imag[0]
;rsi <- real[0]
;rdi <- N

ASMfft:
    mov r8, rdx
    xorpd xmm1, xmm1; k
    xorpd xmm2, xmm2; n
    xorpd xmm3, xmm3; angulo
    xorpd xmm4, xmm4; 2
    xorpd xmm10, xmm10
    mov r12, rdi; N
    mov r13, 0; n  
    mov r14, 0; k
    mov r15, 2; 2
    movsd xmm15, xmm0
    mov r11, 1000000
    

lazo_k:
    xorpd xmm13, xmm13; k_term
    xorpd xmm14, xmm14; k_term_imag 
    jmp lazo_n

lazo_n:
    movsd xmm9, xmm15; xmm9 <- pi 
    movsd xmm3, xmm15; xmm3 <- pi

    cvtsi2sd xmm7, r11;1000000
    cvtsi2sd xmm1, r14; k
    cvtsi2sd xmm2, r13; n 
    cvtsi2sd xmm8, rdi; N
    cvtsi2sd xmm4, r15; 2
;En este caso, xmm1, xmm2,xmm8,xmm4 quedan para nueva asignacion

dividendo:
    mulsd xmm3, xmm1; pi*k
    mulsd xmm3, xmm2; pi*k*n
    mulsd xmm3, xmm4; 2*pi*k*n
    mulsd xmm3, xmm7; 2*pi*k*n*1000000
    divsd xmm3, xmm8; <- 2*pi*k*n*1000000/N


divisor:
    mulsd xmm9, xmm4; 2*pi
    mulsd xmm9, xmm7; 2*pi*1000000
    
    
cociente:
    cvtsd2si rax, xmm3; guardado dividendo en rax
    cvtsd2si rbx, xmm9; guardado divisor en rax
    mov r9, rdx 
    xor rdx, rdx
    div rbx
    mov rdx, r9

cociente_a_flotante:
    cvtsi2sd xmm1, rax
    mulsd xmm1, xmm9
    subsd xmm3, xmm1
    divsd xmm3, xmm7

angulo_xmm3:
    movsd xmm1, xmm3
    movsd xmm3, xmm4

    mov r11, -166130
    mov r10, 8050
    mov r9, -150
    cvtsi2sd xmm2, r11
    cvtsi2sd xmm4, r10
    cvtsi2sd xmm8, r9
    divsd xmm2, xmm7
    divsd xmm4, xmm7
    divsd xmm8, xmm7
    movsd xmm5, xmm8
    movsd xmm12, xmm8

seno_xmm5: 
    

    mulsd xmm5, xmm1
    mulsd xmm5, xmm1
    addsd xmm5, xmm4
    mulsd xmm5, xmm1
    mulsd xmm5, xmm1
    addsd xmm5, xmm2
    mulsd xmm5, xmm1
    mulsd xmm5, xmm1
    mulsd xmm5, xmm1
    addsd xmm5, xmm1

cos_xmm6:
    movsd xmm6, xmm15
    divsd xmm6, xmm3
    movsd xmm8, xmm1
    subsd xmm8, xmm6
    movsd xmm6, xmm8
    
    mulsd xmm12, xmm6
    mulsd xmm12, xmm6
    addsd xmm12, xmm4
    mulsd xmm12, xmm6
    mulsd xmm12, xmm6
    addsd xmm12, xmm2
    mulsd xmm12, xmm6
    mulsd xmm12, xmm6
    mulsd xmm12, xmm6
    addsd xmm12, xmm6

    mov r9, -1
    cvtsi2sd xmm8, r9
    mulsd xmm12, xmm8

;xmm12 es cos, xmm5 es sin
    ;calcular xreal
    movsd xmm7, [rsi];a[n]
    movsd xmm6, [rdx];b[n]
    mulsd xmm7, xmm12
    mulsd xmm6, xmm5
    addsd xmm7, xmm6

    addsd xmm13, xmm7

    ;calcular x_imag
    movsd xmm7, [rsi];a[n]
    movsd xmm6, [rdx];b[n]
    mulsd xmm6, xmm12
    mulsd xmm7, xmm5
    subsd xmm6, xmm7

    
    
    addsd xmm14, xmm6


    cmp r13, r12
    jl aumento_n
    jmp lazo_siguiente_k

lazo_siguiente_k:
    movsd [rcx], xmm13
    movsd [r8], xmm14
    cmp r14, r12
    jl aumento_k
    jmp salida

aumento_k:
    add r14, 1
    jmp lazo_k

aumento_n:
    add r13,1
    jmp lazo_n
    
salida:
    ret



Overwriting ASMfft.asm


In [508]:
! gcc -c -fpic Cfft.c -o cfft.o
! gcc -shared cfft.o -o cfft.so

! nasm -f elf64 ASMfft.asm -o asmfft.o -g
! gcc -shared asmfft.o -o asmfft.so

! python3 mainfft.py

Reales:  15.1
Imaginarios:  5.1
Reales:  15.1
Imaginarios:  5.1
Reales:  1.02156867e-316
Imaginarios:  0.0
[7.10632911e+001 8.15208316e-322 5.81468329e-318 1.02156867e-316
 7.32485615e-316]
[0. 0. 0. 0. 0.]
munmap_chunk(): invalid pointer


#### d) En el programa principal, generar un arreglo de prueba que tenga componentes reales e imaginarios. Posteriormente, separar el arreglo en dos arreglos(uno con componentes reales y otro con componente imaginario). Validar sus funciones comparando con la función fft de la librería numpy o scipy. Considerar elementos tipo float.


In [502]:
%%file mainfft.py
PI = 3.141592653589793
import ctypes
import numpy as np
import random
import math

def fftpy(real,imag):
    N = len(real)
    salida_real = []
    salida_imag = []
    for k in range(N):
        real_acumulado = 0
        imag_acumulado = 0
        for n in range(N):
            angle = 2*math.pi*k*n/N
            real_acumulado += real[n]*math.cos(angle) + imag[n]*math.sin(angle)
            imag_acumulado += imag[n]*math.cos(angle) - real[n]*math.sin(angle)
        salida_real.append(real_acumulado)
        salida_imag.append(imag_acumulado)
    return salida_real, salida_imag

if __name__ == '__main__':
    Lib_C = ctypes.CDLL('./cfft.so')
    Lib_ASM = ctypes.CDLL('./asmfft.so')

    Lib_C.fftC.argtypes = [ctypes.c_int,
        np.ctypeslib.ndpointer(dtype=np.float64),
        np.ctypeslib.ndpointer(dtype=np.float64),
        np.ctypeslib.ndpointer(dtype=np.float64),
        np.ctypeslib.ndpointer(dtype=np.float64),
        ]
    Lib_ASM.ASMfft.argtypes = [ctypes.c_int,
        np.ctypeslib.ndpointer(dtype=np.float64),
        np.ctypeslib.ndpointer(dtype=np.float64),
        np.ctypeslib.ndpointer(dtype=np.float64),
        np.ctypeslib.ndpointer(dtype=np.float64),
        ctypes.c_double
    ]
    x_a = np.array([1.1,2,3,4,5],dtype=np.float64)
    x_b = np.array([1.1,1,1,1,1],dtype=np.float64)
    salida1, salida2 = fftpy(x_a,x_b)
    print('Reales: ',salida1[0])
    print('Imaginarios: ',salida2[0])

    x_a = np.array([1.1,2,3,4,5],dtype=np.float64)
    x_b = np.array([1.1,1,1,1,1],dtype=np.float64)
    x_a1 = np.array([1,2,3,4,5],dtype=np.float64)
    x_b1 = np.array([1,1,1,1,1],dtype=np.float64)
    Lib_C.fftC(5,x_a,x_b,x_a1,x_b1)
    print('Reales: ',x_a1[0])
    print('Imaginarios: ',x_b1[0])

    x_a = np.array([1,2,3,4,5],dtype=np.float64)
    x_b = np.array([1,1,1,1,1],dtype=np.float64)
    x_a1 = np.array([0,0,0,0,0],dtype=np.float64)
    x_b1 = np.array([0,0,0,0,0],dtype=np.float64)
    Lib_ASM.ASMfft(5,x_a,x_b,x_a1,x_b1,PI)
    print('Reales: ',x_a1[3])
    print('Imaginarios: ',x_b1[3])
    print(x_a1)
    print(x_b1)

Overwriting mainfft.py
